In [1]:
from graph_ds import PyH3Graph
from pois_to_h3 import get_pois_h3
from ghsl_processing import get_origins, city_boundaries_to_h3 

import pandas as pd
import h3.api.numpy_int as h3
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as cx
import numpy as np
import seaborn as sns
import pickle


def plot_hex_df(df, markersize, color_column=None, color_categorical=False):
    df[['y','x']] = list(df['h3_index'].apply(h3.h3_to_geo))
    # Convert the pandas DataFrame to a GeoPandas DataFrame with a Point geometry column
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.x, df.y))

    fig, ax = plt.subplots(figsize=(10,10))
    # Show the GeoPandas DataFrame

    if color_column:
        gdf.plot(ax=ax, column=color_column, categorical=color_categorical,
                  legend=True, markersize=markersize, alpha=0.5, cmap='Reds',
                  vmin=0, vmax=25)
    else:
        gdf.plot(ax=ax, markersize=markersize, alpha=0.5)

    #plt.xlim(12.391681,12.737388)
    #plt.ylim(55.549206, 55.759991)

    cx.add_basemap(ax = ax, crs="EPSG:4326")
    #plt.show()

def routing(category_set, origins, destinations, num_origins, graph):
    mins={}
    category_travel_time_means = {}
    for c in category_set:
        cat_destinations = list(destinations[destinations['category']==c]['h3_index'])
        ds = graph.matrix_distance(origins=origins, destinations=cat_destinations, dynamic_infinity=True)

        # they might be different lengths, so we can't use a df
        mins[c]={k:np.nanmin(np.array(v,dtype=np.float64)) for k,v in ds.items() if v}
        category_travel_time_means[c] = np.nanmean(list(mins[c].values()))

    return category_travel_time_means, mins


/home/inkompotato/micromamba/envs/graphs/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
# # denver, cl
# osm_file = "../resources/denver-processed.osm.pbf"
# gtfs_files = ["../resources/denver_gtfs.zip"]
# ghsl_file = "../resources/GHS_BUILT_C_MSZ_E2018_GLOBE_R2022A_54009_10_V1_0_R5_C10.tif"

# city_name = "Denver, Colorado"

# copehagen, dk
# osm_file = "../resources/processed/København Kommune_processed.osm.pbf"
# gtfs_files = ["../resources/rejseplanen.zip"]
# ghsl_file = "../resources/GHS_BUILT_C_MSZ_E2018_GLOBE_R2022A_54009_10_V1_0_R3_C19_Denmark.tif"

# city_name = "København Kommune"

# Los Angeles
osm_file = "../resources/la/la-processed.osm.pbf"
gtfs_files = ["../resources/la/gtfs_bus.zip", "../resources/la/gtfs_rail.zip"]
ghsl_file = "../resources/la/GHS_BUILT_C_MSZ_E2018_GLOBE_R2022A_54009_10_V1_0_R5_C8_SoCal.tif"

city_name = "Los Angeles"

In [3]:
# Parameters

# 12 is around 10 meter resolution
H3_RES = 12

# how many origins to sample
num_origins = 1000

#essential filter will be the least restrictive
essential_filter = {
    "amenity":["pharmacy","dentist","clinic","doctors","school"],
    "shop":["supermarket","greengrocer","medical_supply","grocery","wholesale"],
    "healthcare":["clinic","doctor","pharmacy","dentist"],
    "leisure":["park"], "sport":True
}

# categories considered for the n-minute calculation
category_set = ['pharmacy','park','supermarket','sport', 'school']

# how to map osm tags to categories
osm_tag_mapping = {
    "medical_supply":"pharmacy",
    "greengrocer":"supermarket",
    "wholesale":"supermarket",
    "grocery":"supermarket",
    "clinic":"doctor",
    "doctors":"doctor",
    'pitch':'sport',
    'track':'sport',
    'sports_centre':'sport'
}

In [4]:
results = {}

city_bounds_h3, bbox, bbox_pois = city_boundaries_to_h3([city_name])

origins = get_origins(H3_RES, [city_name], bbox, ghsl_file, city_bounds_h3)
print("origins", len(origins))

origins_sample = list(origins[origins['residential_bool']==1].sample(num_origins, replace=True)['h3_index'])


destinations = get_pois_h3(osm_file, essential_filter, H3_RES, category_set, osm_tag_mapping, [city_name])

for g_type in ['all', 'walk', 'walk+bike', 'walk+transit']:
        print(g_type)
        # build the graph
        graph = PyH3Graph(bike_penalty=1, k_ring=2, layers=g_type)
        graph.create(osm_path=osm_file,gtfs_paths=gtfs_files)
        # do routing
        category_means, route_mins = routing(category_set, origins_sample, destinations, num_origins, graph)
        # save results as dictionary
        results.update({g_type:{'category_means':category_means, 'route_mins':route_mins}})

origins 1555000


/home/inkompotato/micromamba/envs/graphs/lib/python3.10/site-packages/pyrosm/pyrosm.py:576: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  gdf = get_poi_data(
/home/inkompotato/micromamba/envs/graphs/lib/python3.10/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(


all
processing osm pbf file: ../resources/la/la-processed.osm.pbf
converted OSM file into 4518307 edges
osm graph created with 3924641 nodes in 31.185535 s
getting GTFS feed from ../resources/la/gtfs_bus.zip
routes: 115
gtfs graph created with 26524 nodes in 6.2981563 s
merged gtfs graph into osm graph, now has 3944833 nodes, took 25 ms
getting GTFS feed from ../resources/la/gtfs_rail.zip
routes: 7
gtfs graph created with 212 nodes in 0.21095392 s
merged gtfs graph into osm graph, now has 3944980 nodes, took 0 ms
hash: 2652502120673578787


/tmp/ipykernel_253087/4198998382.py:44: RuntimeWarning: All-NaN slice encountered
  mins[c]={k:np.nanmin(np.array(v,dtype=np.float64)) for k,v in ds.items() if v}


walk
processing osm pbf file: ../resources/la/la-processed.osm.pbf
converted OSM file into 4181695 edges
osm graph created with 3611308 nodes in 26.98458 s
hash: 2803992494559127235
walk+bike
processing osm pbf file: ../resources/la/la-processed.osm.pbf
converted OSM file into 4518307 edges
osm graph created with 3924641 nodes in 30.804897 s
hash: 11213364301991448014
walk+transit
processing osm pbf file: ../resources/la/la-processed.osm.pbf
converted OSM file into 4181695 edges
osm graph created with 3611308 nodes in 24.406452 s
getting GTFS feed from ../resources/la/gtfs_bus.zip
routes: 115
gtfs graph created with 26524 nodes in 6.357312 s
merged gtfs graph into osm graph, now has 3631500 nodes, took 29 ms
getting GTFS feed from ../resources/la/gtfs_rail.zip
routes: 7
gtfs graph created with 212 nodes in 0.21512155 s
merged gtfs graph into osm graph, now has 3631647 nodes, took 0 ms
hash: 14015992853808157881


In [5]:
import polars as pl
from lets_plot import *
LetsPlot.setup_html()

In [6]:
result_data = []
for g_type, result in results.items():
    route_mins = result['route_mins']
    for category, values in route_mins.items():
        for h3_index, value in values.items():
            lat, lon = h3.h3_to_geo(h3_index)
            result_data.append([g_type, category, h3_index, lat, lon, value])

result_df = pl.DataFrame(result_data, columns=['graph_type', 'category', 'h3', 'lat', 'lon', 'travel_time'])

result_df.head()

/tmp/ipykernel_253087/865589535.py:9: DeprecationWarning: `columns` is deprecated as an argument to `__init__`; use `schema` instead.
  result_df = pl.DataFrame(result_data, columns=['graph_type', 'category', 'h3', 'lat', 'lon', 'travel_time'])


graph_type,category,h3,lat,lon,travel_time
str,str,i64,f64,f64,f64
"""all""","""pharmacy""",631236328530586111,34.1927,-118.50933,19.399695
"""all""","""pharmacy""",631236348207649791,34.104847,-118.272849,9.95994
"""all""","""pharmacy""",631236328536969727,34.181422,-118.511996,14.319879
"""all""","""pharmacy""",631236332518148607,34.080947,-118.440194,10.003083
"""all""","""pharmacy""",631236595378510847,33.952651,-118.308175,25.538988


In [7]:
plot_df = result_df.filter((pl.col('travel_time').is_not_nan()) & (pl.col('travel_time') < 45.0))

In [8]:
ggplot(plot_df) + \
    geom_livemap() + \
    geom_point(aes(x='lon', y='lat'), size=0.5) + \
    ggtitle(f"Origins in {city_name}")

In [9]:
ggplot(plot_df) + \
    geom_point(aes(x='lon', y='lat', color='travel_time'), alpha=0.5) + \
    scale_color_gradient(low='#f2f0f7', high='#54278f') + \
    facet_wrap(["graph_type", "category"], ncol=5) + \
    coord_fixed() + \
    ggtitle(f"Minimum travel times to essential services in [{city_name}]") + \
    ggsize(1000, 700)

In [10]:
ggplot(plot_df) + \
    geom_density(aes(x='travel_time', fill='category'), alpha=0.3) + \
    xlim(0, 30) + \
    facet_wrap(["graph_type"], ncol=5) + \
    ggtitle(f"Minimum travel time to essential services in [{city_name}], grouped by network type")

In [11]:
benefit_df = plot_df.groupby(['h3', 'category']).agg(
    pl.col('travel_time'),
    pl.col('travel_time').len().alias('count'),
).filter(pl.col('count') == 4).groupby(['h3', 'category']).agg(
    (pl.col('travel_time').flatten().take(1) - pl.col('travel_time').flatten().take(2)).alias('bike_benefit'),
    (pl.col('travel_time').flatten().take(1) - pl.col('travel_time').flatten().take(3)).alias('transit_benefit'),
    (pl.col('travel_time').flatten().take(1) - pl.col('travel_time').flatten().take(0)).alias('multimodal_benefit'),
)

In [12]:
ggplot(benefit_df) + \
    geom_histogram(aes(x='multimodal_benefit', fill='category'), alpha=0.5, bins=40) + \
    xlim(0, 15) + \
    ggtitle(f"Multimodal travel time benefit over walking in [{city_name}]")

In [13]:
benefit_df.head()

h3,category,bike_benefit,transit_benefit,multimodal_benefit
i64,str,f64,f64,f64
631236595226201599,"""school""",0.0,0.0,0.0
631236328661630463,"""school""",3.556858,0.0,3.556858
631236348413799935,"""school""",0.044993,0.0,0.044993
631236326471585279,"""school""",0.0,0.0,0.0
631236327842983423,"""school""",0.0,0.0,0.0


In [14]:
benefit_df.groupby("category").agg(
    pl.col('bike_benefit').mean().alias('bike_benefit'),    
    pl.col('transit_benefit').mean().alias('transit_benefit'),
    pl.col('multimodal_benefit').mean().alias('multimodal_benefit'),
)

category,bike_benefit,transit_benefit,multimodal_benefit
str,f64,f64,f64
"""pharmacy""",4.263537,2.022191,4.742625
"""supermarket""",2.717628,0.570536,2.805695
"""park""",1.207884,0.106843,1.209067
"""school""",0.362582,0.018322,0.362392
"""sport""",5.109949,2.034438,5.442991


In [15]:
plot_df.head()

graph_type,category,h3,lat,lon,travel_time
str,str,i64,f64,f64,f64
"""all""","""pharmacy""",631236328530586111,34.1927,-118.50933,19.399695
"""all""","""pharmacy""",631236348207649791,34.104847,-118.272849,9.95994
"""all""","""pharmacy""",631236328536969727,34.181422,-118.511996,14.319879
"""all""","""pharmacy""",631236332518148607,34.080947,-118.440194,10.003083
"""all""","""pharmacy""",631236595378510847,33.952651,-118.308175,25.538988


In [16]:
plot_df.groupby(["graph_type", "h3"]).agg(
    pl.col('travel_time').max().alias('max_travel_time')
).groupby("graph_type").agg(
    pl.col('max_travel_time').mean().alias('mean_max_travel_time')
)

graph_type,mean_max_travel_time
str,f64
"""walk+bike""",20.794319
"""walk+transit""",23.948962
"""all""",20.06522
"""walk""",25.683372
